In [1]:
import warnings

# warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import copy
import json
import pandas as pd
import os
import sys
import featuretools as ft
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

sys.path.append("../../")
from Trane import trane as trane
from datetime import datetime, timedelta

### Upload of the dataset and metadata

In [2]:
df = pd.read_csv("./chicago-bike/data/bike-sampled.csv", sep=",")
df["date"] = df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df = df.sort_values(by=["date"])
df = df.fillna(0)
meta = trane.TableMeta(json.loads(open("./chicago-bike/data/meta.json").read()))

In [3]:
df

,date,hour,usertype,gender,tripduration,temperature,from_station_id,dpcapacity_start,to_station_id,dpcapacity_end
103805,2017-01-02,0,Subscriber,Male,10.900000,30.0,130,15.0,119,19.0
102617,2017-01-02,15,Subscriber,Male,5.766667,37.9,332,19.0,153,19.0
102618,2017-01-02,15,Subscriber,Female,18.550000,37.9,5,23.0,176,27.0
102619,2017-01-02,15,Subscriber,Male,7.166667,37.9,313,19.0,340,15.0
102620,2017-01-02,15,Subscriber,Male,2.883333,37.9,84,19.0,133,27.0
...,...,...,...,...,...,...,...,...,...,...
3304,2017-01-31,10,Subscriber,Female,6.966667,37.9,247,19.0,247,19.0
3305,2017-01-31,10,Subscriber,Male,6.483333,37.9,425,15.0,426,19.0
3306,2017-01-31,10,Subscriber,Female,8.250000,37.9,175,19.0,45,15.0
3299,2017-01-31,10,Subscriber,Male,16.266667,37.9,202,15.0,317,23.0


### Defining entity column, time column and cutoff strategy

In [4]:
entity = "usertype"
time = "date"
cutoff = "1h"
cutoff_base = pd.Timestamp(datetime.strptime("2017-01-02", "%Y-%m-%d"))
cutoff_end = pd.Timestamp(datetime.strptime("2017-01-31", "%Y-%m-%d"))
cutoff_strategy = trane.CutoffStrategy(entity, cutoff, cutoff_base, cutoff_end)

problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity, cutoff_strategy=cutoff_strategy, time_col=time
)

problems = problem_generator.generate(df, generate_thresholds=True)

  0%|          | 0/1702 [00:00<?, ?it/s]

Success/Attempt = 735/1702


### Generating prediction problems

In [5]:
problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity, time_col=time, cutoff_strategy=cutoff_strategy
)

problems = problem_generator.generate(df, generate_thresholds=True)

  0%|          | 0/1702 [00:00<?, ?it/s]

Success/Attempt = 735/1702


### Labeling the prediction tasks

In [6]:
for p in problems:
    print(str(p))

For each <usertype> predict the number of records in next 1h days
For each <usertype> predict the number of records with <tripduration> greater than 13.05 in next 1h days
For each <usertype> predict the number of records with <tripduration> greater than 8.033333333333333 in next 1h days
For each <usertype> predict the number of records with <tripduration> greater than 5.116666666666666 in next 1h days
For each <usertype> predict the number of records with <temperature> greater than 37.9 in next 1h days
For each <usertype> predict the number of records with <temperature> greater than 34.0 in next 1h days
For each <usertype> predict the number of records with <temperature> greater than 26.1 in next 1h days
For each <usertype> predict the number of records with <dpcapacity_start> greater than 27.0 in next 1h days
For each <usertype> predict the number of records with <dpcapacity_start> greater than 19.0 in next 1h days
For each <usertype> predict the number of records with <dpcapacity_sta

In [ ]:
for p in problems:
    try:
        x = p.execute(df, -1)
        problem_label_dict[str(p)] = x
    except:
        pass